In [36]:
import pandas as pd

In [37]:
dete = pd.read_csv('dete_survey.csv')
tafe = pd.read_csv('tafe_survey.csv')

# Questions to Answer
* Are employees who only worked for the institutes for a short period of time resigning due to some kind of dissatisfaction?     
    * What about employees who have been there longer?
* Are younger employees resigning due to some kind of dissatisfaction? What about older employees?

### Initial Analysis

In [38]:
dete.isnull().sum()
# dete.shape #(822, 56)

ID                                       0
SeparationType                           0
Cease Date                               0
DETE Start Date                          0
Role Start Date                          0
Position                                 5
Classification                         367
Region                                   0
Business Unit                          696
Employment Status                        5
Career move to public sector             0
Career move to private sector            0
Interpersonal conflicts                  0
Job dissatisfaction                      0
Dissatisfaction with the department      0
Physical work environment                0
Lack of recognition                      0
Lack of job security                     0
Work location                            0
Employment conditions                    0
Maternity/family                         0
Relocation                               0
Study/Travel                             0
Ill Health 

In [39]:
# drop columns where more than half of the values are null
dete.dropna(thresh=411, axis=1, inplace=True)
dete.isnull().sum()
dete.head()

,ID,SeparationType,Cease Date,DETE Start Date,Role Start Date,Position,Classification,Region,Employment Status,Career move to public sector,...,Feedback,Further PD,Communication,My say,Information,Kept informed,Wellness programs,Health & Safety,Gender,Age
0,1,Ill Health Retirement,08/2012,1984,2004,Public Servant,A01-A04,Central Office,Permanent Full-time,True,...,A,A,N,A,A,N,N,N,Male,56-60
1,2,Voluntary Early Retirement (VER),08/2012,Not Stated,Not Stated,Public Servant,AO5-AO7,Central Office,Permanent Full-time,False,...,A,A,N,A,A,N,N,N,Male,56-60
2,3,Voluntary Early Retirement (VER),05/2012,2011,2011,Schools Officer,NaN,Central Office,Permanent Full-time,False,...,N,N,A,A,N,N,N,N,Male,61 or older
3,4,Resignation-Other reasons,05/2012,2005,2006,Teacher,Primary,Central Queensland,Permanent Full-time,False,...,A,A,A,A,A,A,N,A,Female,36-40
4,5,Age Retirement,05/2012,1970,1989,Head of Curriculum/Head of Special Education,NaN,South East,Permanent Full-time,False,...,SA,SA,D,D,A,N,A,M,Female,61 or older


In [40]:
tafe.isnull().sum()
tafe.columns
# tafe.head()

Index(['Record ID', 'Institute', 'WorkArea', 'CESSATION YEAR',
       'Reason for ceasing employment',
       'Contributing Factors. Career Move - Public Sector ',
       'Contributing Factors. Career Move - Private Sector ',
       'Contributing Factors. Career Move - Self-employment',
       'Contributing Factors. Ill Health',
       'Contributing Factors. Maternity/Family',
       'Contributing Factors. Dissatisfaction',
       'Contributing Factors. Job Dissatisfaction',
       'Contributing Factors. Interpersonal Conflict',
       'Contributing Factors. Study', 'Contributing Factors. Travel',
       'Contributing Factors. Other', 'Contributing Factors. NONE',
       'Main Factor. Which of these was the main factor for leaving?',
       'InstituteViews. Topic:1. I feel the senior leadership had a clear vision and direction',
       'InstituteViews. Topic:2. I was given access to skills training to help me do my job better',
       'InstituteViews. Topic:3. I was given adequate oppo

In [41]:
'''
Remove columns with too many nulls they are virtually useless
'''
dete = dete.drop(dete.iloc[:, 28:49], axis=1)
dete.shape

tafe = tafe.drop(tafe.iloc[:, 17:66], axis=1)
tafe.shape

(702, 23)

In [42]:
# make column names uniform
dete.columns = dete.columns.str.lower().str.strip().str.replace(' ', '_')
dete.columns

Index(['id', 'separationtype', 'cease_date', 'dete_start_date',
       'role_start_date', 'position', 'classification', 'region',
       'employment_status', 'career_move_to_public_sector',
       'career_move_to_private_sector', 'interpersonal_conflicts',
       'job_dissatisfaction', 'dissatisfaction_with_the_department',
       'physical_work_environment', 'lack_of_recognition',
       'lack_of_job_security', 'work_location', 'employment_conditions',
       'maternity/family', 'relocation', 'study/travel', 'ill_health',
       'traumatic_incident', 'work_life_balance', 'workload',
       'none_of_the_above', 'professional_development', 'age'],
      dtype='object')

In [43]:
new_tafe_col_names = {
    'Record ID': 'id',
    'CESSATION YEAR': 'cease_date',
    'Reason for ceasing employment': 'separationtype',
    'Gender. What is your Gender?': 'gender',
    'CurrentAge. Current Age': 'age',
    'Employment Type. Employment Type': 'employment_status',
    'Classification. Classification': 'position',
    'LengthofServiceOverall. Overall Length of Service at Institute (in years)': 'institute_service',
    'LengthofServiceCurrent. Length of Service at current workplace (in years)': 'role_service'
}

tafe = tafe.rename(columns=new_tafe_col_names).copy(deep=True)
tafe.columns = tafe.columns.str.replace('Contributing Factors.', '').str.lower().str.strip().str.replace('- ', '_').str.replace(' ', '_')
tafe.columns

Index(['id', 'institute', 'workarea', 'cease_date', 'separationtype',
       'career_move__public_sector', 'career_move__private_sector',
       'career_move__self-employment', 'ill_health', 'maternity/family',
       'dissatisfaction', 'job_dissatisfaction', 'interpersonal_conflict',
       'study', 'travel', 'other', 'none', 'gender', 'age',
       'employment_status', 'position', 'institute_service', 'role_service'],
      dtype='object')

In [44]:
# remove repeated phrases in column names
# it might be more efficient to remove everything that comes before a period in a column name, instead of replacing them one by one.
# tafe.columns[tafe.columns.str.find('.') != -1] # find columns with a period, as they have repeated phrases

In [45]:
dete['separationtype'].unique()
tafe['separationtype'].unique()

array(['Contract Expired', 'Retirement', 'Resignation',
       'Retrenchment/ Redundancy', 'Termination', 'Transfer', nan],
      dtype=object)

In [46]:
'''
Create dataframes containing records where dete['separationtype'] and tafe['reason_for_ceasing_employment'] are 'Resignation'
'''
dete_resignations = dete[dete['separationtype'].str.find('Resignation') != -1].copy(deep=True)
tafe_resignations = tafe[tafe['separationtype'].str.find('Resignation') != -1].copy(deep=True)

In [47]:
'''
Format the year column appropriately, and extract year from mm/yyyy combination
'''
# interesting finding 'astype(int)' does not work below
tafe_resignations['cease_date'] = tafe_resignations['cease_date'].astype("Int64")
# tafe_resignations.head()

# convenient way to extract year from 'mm/yyyy' is to get the four consecutive digits
dete_resignations['cease_date'] = dete_resignations['cease_date'].str.extract(pat='(\d{4})')
# dete_resignations['cease_date']

### Combine Columns
Instead of having values scattered across different columns

In [48]:
'''
Some columns have complementary and exclusive values, meaning where one is populated the others are nulls.
Thus, these may be combined into one column since the values are appropriately named, and the others done away with.
'''
# some values have a '-' instead of None, which is affecting the working of the 'fillna()' method
tafe_resignations = tafe_resignations.replace(to_replace='-', value=None).copy(deep=True)

# 'fillna() from one column with values from the other columns
tafe_resignations['career_move__public_sector'] = tafe_resignations['career_move__public_sector'].fillna(value=tafe_resignations['career_move__private_sector'])
tafe_resignations['career_move__public_sector'] = tafe_resignations['career_move__public_sector'].fillna(value=tafe_resignations['career_move__self-employment'])

# drop the other columns now that their values have been recorded in the one we'll be keeeping
tafe_resignations = tafe_resignations.drop(columns=['career_move__private_sector', 'career_move__self-employment']).copy(deep=True)

# rename the main column to reflect all the data appropriately
tafe_resignations = tafe_resignations.rename(columns={'career_move__public_sector': 'career_move_to'}).copy(deep=True)
# edit column values to remove content already captured in the column name
tafe_resignations['career_move_to'] = tafe_resignations['career_move_to'].str.replace('Career Move -', '').str.strip()
tafe_resignations

,id,institute,workarea,cease_date,separationtype,career_move_to,ill_health,maternity/family,dissatisfaction,job_dissatisfaction,...,study,travel,other,none,gender,age,employment_status,position,institute_service,role_service
3,6.341399e+17,Mount Isa Institute of TAFE,Non-Delivery (corporate),2010,Resignation,None,None,None,None,None,...,None,Travel,None,None,NaN,NaN,NaN,NaN,NaN,NaN
4,6.341466e+17,Southern Queensland Institute of TAFE,Delivery (teaching),2010,Resignation,Private Sector,None,None,None,None,...,None,None,None,None,Male,41 45,Permanent Full-time,Teacher (including LVT),3-4,3-4
5,6.341475e+17,Southern Queensland Institute of TAFE,Delivery (teaching),2010,Resignation,None,None,None,None,None,...,None,None,Other,None,Female,56 or older,Contract/casual,Teacher (including LVT),7-10,7-10
6,6.341520e+17,Barrier Reef Institute of TAFE,Non-Delivery (corporate),2010,Resignation,Private Sector,None,Maternity/Family,None,None,...,None,None,Other,None,Male,20 or younger,Temporary Full-time,Administration (AO),3-4,3-4
7,6.341537e+17,Southern Queensland Institute of TAFE,Delivery (teaching),2010,Resignation,None,None,None,None,None,...,None,None,Other,None,Male,46 50,Permanent Full-time,Teacher (including LVT),3-4,3-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,6.350660e+17,Southern Queensland Institute of TAFE,Non-Delivery (corporate),2013,Resignation,Private Sector,None,None,None,None,...,None,None,None,None,Male,21 25,Temporary Full-time,Operational (OO),5-6,5-6
697,6.350668e+17,Barrier Reef Institute of TAFE,Delivery (teaching),2013,Resignation,Public Sector,None,None,None,None,...,None,None,None,None,Male,51-55,Temporary Full-time,Teacher (including LVT),1-2,1-2
698,6.350677e+17,Southern Queensland Institute of TAFE,Non-Delivery (corporate),2013,Resignation,Public Sector,None,None,None,None,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
699,6.350704e+17,Tropical North Institute of TAFE,Delivery (teaching),2013,Resignation,None,None,None,None,None,...,None,None,Other,None,Female,51-55,Permanent Full-time,Teacher (including LVT),5-6,1-2


In [49]:
tafe_resignations.columns

Index(['id', 'institute', 'workarea', 'cease_date', 'separationtype',
       'career_move_to', 'ill_health', 'maternity/family', 'dissatisfaction',
       'job_dissatisfaction', 'interpersonal_conflict', 'study', 'travel',
       'other', 'none', 'gender', 'age', 'employment_status', 'position',
       'institute_service', 'role_service'],
      dtype='object')

### Reasons Columns
The following columns may be summed into a single column 'reason' once it is determine their values do not collide
* ['ill_health', 'maternity/family', 'dissatisfaction', 'job_dissatisfaction', 'interpersonal_conflict', 'study', 'travel', 'other', 'none']

In [50]:
tafe_resignations[['ill_health', 'maternity/family', 'dissatisfaction', 'job_dissatisfaction', 'interpersonal_conflict', 'study', 'travel', 'other', 'none']].isnull().sum()

ill_health                320
maternity/family          321
dissatisfaction           286
job_dissatisfaction       279
interpersonal_conflict    317
study                     325
travel                    324
other                     255
none                      325
dtype: int64

In [51]:
dete_resignations[['career_move_to_public_sector',
       'career_move_to_private_sector', 'interpersonal_conflicts',
       'job_dissatisfaction', 'dissatisfaction_with_the_department',
       'physical_work_environment', 'lack_of_recognition',
       'lack_of_job_security', 'work_location', 'employment_conditions',
       'maternity/family', 'relocation', 'study/travel', 'ill_health',
       'traumatic_incident', 'work_life_balance', 'workload',
       'none_of_the_above', 'professional_development', 'age']]
# dete_resignations.columns

,career_move_to_public_sector,career_move_to_private_sector,interpersonal_conflicts,job_dissatisfaction,dissatisfaction_with_the_department,physical_work_environment,lack_of_recognition,lack_of_job_security,work_location,employment_conditions,maternity/family,relocation,study/travel,ill_health,traumatic_incident,work_life_balance,workload,none_of_the_above,professional_development,age
3,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,A,36-40
5,False,True,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,SD,41-45
8,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,D,31-35
9,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,SD,46-50
11,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,N,31-35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,A,26-30
815,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,A,21-25
816,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,SA,21-25
819,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,True,False,False,A,31-35


In [52]:
reasons_columns = ['career_move_to_public_sector',
       'career_move_to_private_sector', 'interpersonal_conflicts',
       'job_dissatisfaction', 'dissatisfaction_with_the_department',
       'physical_work_environment', 'lack_of_recognition',
       'lack_of_job_security', 'work_location', 'employment_conditions',
       'maternity/family', 'relocation', 'study/travel', 'ill_health',
       'traumatic_incident', 'work_life_balance', 'workload',
       'none_of_the_above']

'''
To condense data that is across columns into one, change values from Boolean to recognize them later.
Since reasons columns have values True or False, change True to column name, and False  to None, so as to use 'fillna()' later
'''

for col in reasons_columns:
    dete_resignations[col] = dete_resignations[col].replace(to_replace={True: col, False: None}).copy(deep=True)

### Combine Columns
Do we have a row with both 'career_move_to_public_sector' and 'career_move_to_private_sector' populated?    
* No    
Thus, we may 'fillna()' one column with the other

In [54]:
dete_resignations[~dete_resignations['career_move_to_public_sector'].isnull() & 
                  ~dete_resignations['career_move_to_private_sector'].isnull()][['career_move_to_public_sector', 'career_move_to_private_sector']]

,career_move_to_public_sector,career_move_to_private_sector


In [55]:
dete_resignations['career_move_to_public_sector'] = dete_resignations['career_move_to_public_sector'].fillna(
    value=dete_resignations['career_move_to_private_sector']).copy(deep=True)
dete_resignations['career_move_to_public_sector']

3      career_move_to_private_sector
5      career_move_to_private_sector
8      career_move_to_private_sector
9                               None
11                              None
                   ...              
808                             None
815    career_move_to_private_sector
816                             None
819                             None
821                             None
Name: career_move_to_public_sector, Length: 311, dtype: object

In [57]:
'''
Data from 'private_sector' is now captured in 'career_move_to_public_sector', drop 'career_move_to_private_sector'.
Since 'career_move_to_public_sector' is no longer capturing only public sector, rename to 'career_move_to'.
'''
dete_resignations = dete_resignations.drop('career_move_to_private_sector', axis=1).copy(deep=True)

dete_resignations = dete_resignations.rename(columns={'career_move_to_public_sector': 'career_move_to'}).copy()

In [58]:
# Remove repeated words in the column content.

dete_resignations['career_move_to'] = dete_resignations['career_move_to'].str.replace('career_move_to_', '').copy()
dete_resignations['career_move_to']

3      private_sector
5      private_sector
8      private_sector
9                None
11               None
            ...      
808              None
815    private_sector
816              None
819              None
821              None
Name: career_move_to, Length: 311, dtype: object

In [59]:
dete_resignations[['career_move_to', 'interpersonal_conflicts',
       'job_dissatisfaction', 'dissatisfaction_with_the_department',
       'physical_work_environment', 'lack_of_recognition',
       'lack_of_job_security', 'work_location', 'employment_conditions',
       'maternity/family', 'relocation', 'study/travel', 'ill_health',
       'traumatic_incident', 'work_life_balance', 'workload'
       ]]

,career_move_to,interpersonal_conflicts,job_dissatisfaction,dissatisfaction_with_the_department,physical_work_environment,lack_of_recognition,lack_of_job_security,work_location,employment_conditions,maternity/family,relocation,study/travel,ill_health,traumatic_incident,work_life_balance,workload
3,private_sector,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,private_sector,None,None,None,None,None,None,None,employment_conditions,maternity/family,None,None,None,None,None,None
8,private_sector,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
9,None,interpersonal_conflicts,job_dissatisfaction,dissatisfaction_with_the_department,None,None,None,None,None,None,None,None,None,None,None,None
11,None,None,None,None,None,None,None,None,None,maternity/family,relocation,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808,None,None,None,None,None,None,None,None,None,maternity/family,None,None,None,None,None,None
815,private_sector,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
816,None,None,None,None,None,None,None,None,None,None,None,study/travel,None,None,None,None
819,None,None,None,None,None,None,None,None,None,maternity/family,relocation,None,None,None,work_life_balance,None


### Other Reasons
Put these values together in the same column, as oppossed to having them scattered across many.

In [27]:
# dete_resignations = dete_resignations.rename(columns={'interpersonal_conflicts': 'other_reasons'}).copy()

In [31]:
dete_resignations['institute_service'] = None

In [95]:
# dete_resignations['institute_service'] = dete_resignations['cease_date'].astype("Int64") - dete_resignations['dete_start_date'].astype("Int64")

In [102]:
# tafe_resignations['dissatisfaction'].unique()
tafe_resignations['job_dissatisfaction'].unique()

array([None, 'Job Dissatisfaction', nan], dtype=object)

In [137]:
tafe_resignations['dissatisfaction'].unique()

array([None, 'Contributing Factors. Dissatisfaction ', nan], dtype=object)

### Guided Project not Complete
I skipped some steps due to hiccups I encountered proceeding,     
I could not find a way to combine values across different columns in one.
This shall be revisited.